<a href="https://colab.research.google.com/github/alyssapinnock/HW2-EEL4810/blob/main/hw2_eel4810.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports needed for the model and testing

In [1]:
# main imports
import torch
import torch.nn as nn
import torch.optim as optim #import optimizers
from torch.utils.data import DataLoader, TensorDataset

# data processing imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# visualization imports
import matplotlib.pyplot as plt
%matplotlib inline

from google.colab import files #file uploads
import zipfile
import os


import warnings
warnings.filterwarnings('ignore') #to suppress warnings

Uploading the files and parsing to grab the features and labels

In [8]:
all_features = []
all_labels = []

# loops to parse and collect the data for features and labels
for root, dirs, files in os.walk('/content/dataset'):
  for file in files:
    if file.endswith('.csv'):
      df = pd.read_csv(os.path.join(root, file), header=None)
      df = df.apply(pd.to_numeric, errors='coerce').dropna()
      if len(df) > 0:
        all_features.append(df.iloc[:, 1:4].values) # parses columns 2-4 and sets as features
        all_labels.append(df.iloc[:, 4].values) # parses column 5 and sets as labels

X = torch.FloatTensor(np.vstack(all_features))
y = torch.FloatTensor(np.hstack(all_labels)).reshape(-1, 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)


Developing and training the model



In [11]:
# MODEL DEVELOPMENT #
class NeuralNetworkModel(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1)

        )

    def forward(self, x):
        return self.layers(x)


# MODEL TRAINING #
model = NeuralNetworkModel(input_size=X_train.shape[1])
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
epochs = 200 # increased epochs from 100 to 200 for longer training time

for epoch in range(epochs):
  optimizer.zero_grad()
  outputs = model(X_train)
  loss = criterion(outputs, y_train)
  loss.backward()
  optimizer.step()
  if(epoch + 1) % 10 == 0:
    print(f'Epoch {epoch+1}, Loss: {loss.item():.4f}')

model.eval()
with torch.no_grad():
  test_loss = criterion(model(X_test), y_test)
  print(f'Test loss: {test_loss.item():.4f}')

Epoch 10, Loss: 0.4184
Epoch 20, Loss: 0.3614
Epoch 30, Loss: 0.3475
Epoch 40, Loss: 0.3231
Epoch 50, Loss: 0.3119
Epoch 60, Loss: 0.3041
Epoch 70, Loss: 0.2966
Epoch 80, Loss: 0.2891
Epoch 90, Loss: 0.2821
Epoch 100, Loss: 0.2756
Epoch 110, Loss: 0.2709
Epoch 120, Loss: 0.2668
Epoch 130, Loss: 0.2631
Epoch 140, Loss: 0.2602
Epoch 150, Loss: 0.2578
Epoch 160, Loss: 0.2556
Epoch 170, Loss: 0.2533
Epoch 180, Loss: 0.2523
Epoch 190, Loss: 0.2514
Epoch 200, Loss: 0.2526
Test loss: 0.2640


Experiments (Part 2)